In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", temperature=0)
reponse = llm.invoke("Hola como estas?")
reponse.content

'¡Hola! Estoy aquí y listo para ayudarte. ¿En qué puedo asistirte hoy?'

In [3]:
response = llm.invoke("Dime los productos que ofreces en la tienda")
response.content

'No tengo una tienda física ni ofrezco productos, pero puedo ayudarte a crear una lista de productos que podrías ofrecer en una tienda, o darte ideas sobre qué tipo de productos podrían ser populares. ¿Te gustaría que te ayude con eso?'

In [4]:
system_prompt = """
Eres un asistente de ventas que ayuda a los clientes a encontrar los productos que buscan.

Y tus productos son:
- Computadoras
- Tablets
- Celulares
- Accesorios
"""

messages = [
    ("system", system_prompt),
    ("user", "Hola, que productos ofrece la tienda?")
]

response = llm.invoke(messages)
response.content

'¡Hola! En nuestra tienda ofrecemos una variedad de productos, que incluyen:\n\n- Computadoras\n- Tablets\n- Celulares\n- Accesorios\n\nSi estás interesado en alguno de estos productos o necesitas ayuda para encontrar algo específico, ¡estaré encantado de ayudarte!'

In [11]:
from langchain_core.tools import tool
import requests

@tool("get_products", description="Busca los productos que coinciden con la consulta")
def get_products(price: int):
    """
    Busca los productos que coinciden con la consulta
    """
    products = [
        {"name": "Computadora", "price": 1000},
        {"name": "Tablet", "price": 500},
        {"name": "Celular", "price": 200},
        {"name": "Accesorio", "price": 100}
    ]
    return "".join([f"{product['name']} - {product['price']}" for product in products if product['price'] == price])


In [12]:
get_products.invoke({"price": 100})

'Accesorio - 100'

In [13]:
from langchain_core.tools import tool
import requests

@tool("get_products", description="Busca los productos que coinciden con la consulta")
def get_products():
    """
    Busca los productos que coinciden con la consulta
    """
    products = [
        {"name": "Computadora", "price": 1000},
        {"name": "Tablet", "price": 500},
        {"name": "Celular", "price": 200},
        {"name": "Accesorio", "price": 100}
    ]
    return "".join([f"{product['name']} - {product['price']}" for product in products])

In [14]:
get_products.invoke({})

'Computadora - 1000Tablet - 500Celular - 200Accesorio - 100'

In [15]:
from langchain_core.tools import tool
import requests

@tool("get_products", description="Busca los productos que coinciden con la consulta")
def get_products():
    # Connect with API
    """
    Busca los productos que coinciden con la consulta
    """
    url = "https://api.escuelajs.co/api/v1/products"
    response = requests.get(url)
    products = response.json()
    return "".join([f"{product['title']} - {product['price']}" for product in products])

In [16]:
get_products.invoke({})

'Classic White Tee - Timeless Style and Comfort - 73Classic Black T-Shirt - 35Sleek White & Orange Wireless Gaming Controller - 69Sleek Wireless Headphone & Inked Earbud Set - 44Sleek Comfort-Fit Over-Ear Headphones - 28Efficient 2-Slice Toaster - 48Sleek Wireless Computer Mouse - 10Sleek Modern Laptop for Professionals - 97Stylish Red & Silver Over-Ear Headphones - 39Sleek Mirror Finish Phone Case - 27Sleek Smartwatch with Vibrant Display - 16Sleek Modern Leather Sofa - 53Mid-Century Modern Wooden Dining Table - 24Elegant Golden-Base Stone Top Dining Table - 66Modern Elegance Teal Armchair - 25Elegant Solid Wood Dining Table - 67Modern Minimalist Workstation Setup - 49Modern Ergonomic Office Chair - 71Futuristic Holographic Soccer Cleats - 39Chic Summer Denim Espadrille Sandals - 33Vibrant Runners: Bold Orange & Blue Sneakers - 27Vibrant Pink Classic Sneakers - 84Futuristic Silver and Gold High-Top Sneaker - 68Futuristic Chic High-Heel Boots - 36Elegant Patent Leather Peep-Toe Pumps w

In [17]:
from typing import Optional, List, Dict, Any
from pydantic import BaseModel, Field, confloat, conint

class SearchProductsArgs(BaseModel):

    """Parámetros de búsqueda (se filtran client-side)."""

    q: Optional[str] = Field(
        default=None,
        description="Texto a buscar dentro del título o descripción (case-insensitive)."
    )

    category_name: Optional[str] = Field(
        default=None,
        description="Nombre de categoría exacto (ej. 'Clothes')."
    )

    min_price: Optional[confloat(ge=0)] = Field(
        default=None,
        description="Precio mínimo."
    )

    max_price: Optional[confloat(ge=0)] = Field(
        default=None,
        description="Precio máximo."
    )

    limit: Optional[conint(ge=1, le=100)] = Field(
        default=10,
        description="Cantidad máxima de items a devolver (1..100)."
    )

# =========================
# Tool
# =========================

@tool(
    "busca_productos_platzi",
    args_schema=SearchProductsArgs,
    description=(
        "Busca productos en la API pública de Platzi Store y devuelve una lista de "
        "objetos con 'title', 'category' y 'price'. Aplica filtros client-side."  #Esta descripcion hace la diferencia para que el agente use la tool correctamente
    ),
)

def busca_productos_platzi(
    q: Optional[str] = None,
    category_name: Optional[str] = None,
    min_price: Optional[float] = None,
    max_price: Optional[float] = None,
    limit: int = 10,

) -> List[Dict[str, Any]]:

    """
    Devuelve una lista de dicts: [{"title": str, "category": str, "price": float}, ...]
    """

    url = "https://api.escuelajs.co/api/v1/products"

    try:
        resp = requests.get(url, timeout=15)
        resp.raise_for_status()
        products = resp.json()

    except requests.RequestException as e:
        return [{"title": "ERROR", "category": "N/A", "price": -1, "detail": str(e)}]

# Para probarlo out3 = 

busca_productos_platzi.invoke({
    "q": "tee",
    "min_price": 50,
    "limit": 5
})